In [23]:
# !pip install nltk

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [19]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

input_text = "Today is sure to be a difficult day because it is raining and the educational content is boring"
# target_text = "I was bored all day today"
# target_text = "Today's class was boring"
target_text = "Today's class was exciting"

tokens = tokenizer(input_text, target_text, return_tensors="pt")
# type(tokens), tokens

result = model(**tokens)
torch.softmax(result.logits, dim=1)

tensor([[0.8902, 0.1098]], grad_fn=<SoftmaxBackward0>)

In [29]:
# nltk.download("movie_reviews")
fieldids = movie_reviews.fileids()
reviews = [ movie_reviews.raw(fieldid) for fieldid in fieldids ]
categories = [ movie_reviews.categories(fieldid) for fieldid in fieldids ]

In [34]:
type(categories[:5]), np.unique(categories, return_counts=True), categories[:5], reviews[:5]

(list,
 (array(['neg', 'pos'], dtype='<U3'), array([1000, 1000], dtype=int64)),
 [['neg'], ['neg'], ['neg'], ['neg'], ['neg']],
 ['plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s s

In [37]:
labeled_categories = LabelEncoder().fit_transform(categories)
labeled_categories

c:\ProgramData\miniconda3\envs\study-env2\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [41]:
X_train, X_test ,y_train, y_test = \
    train_test_split(reviews, labeled_categories, stratify=labeled_categories, random_state=42)

In [44]:
y_train

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

In [49]:
device_name = "cuda:0" if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
device

In [72]:
tokenizer2 = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model2 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

model2.to(device)

batch_size = 10
all_prediction_labels = []
for idx in range(y_train.shape[0]//batch_size):
    input_texts = X_train[idx*batch_size:(idx+1)*batch_size]
    inputs = tokenizer2(input_texts, padding=True, truncation=True, return_tensors="pt")
    inputs = inputs.to(device)
    result = model2(**inputs) # 예측
    predictions = torch.softmax(result.logits, dim=1)
    prediction_labels = predictions.cpu().detach().numpy().argmax(axis=1)
    all_prediction_labels.extend(prediction_labels.tolist())


In [77]:
accuracy = (all_prediction_labels == y_train).sum() / y_train.shape[0]
print(f"train accuracy : {accuracy}")

train accuracy : 0.846
